In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [ ]:
# convert to a format amenable for http://www.ncbi.nlm.nih.gov/genome/tools/remap
df_d = pd.read_csv('decipher_list.txt', sep='\t')
with open('decipher_list_ucsc_format.txt','w')as deci:
    for i in xrange(len(df_d)):
        
        if df_d.iloc[i,1] == 23:
            chrm = 'X'
        elif df_d.iloc[i,1] == 24:
            chrm = 'Y'
        else:
            chrm = df_d.iloc[i,1]
        string = str(chrm) + '\t' + str(df_d.iloc[i,2]) + '\t' + str(df_d.iloc[i,3]) + '\n'
        deci.write(string)

In [35]:
# convert from outputted format from http://www.ncbi.nlm.nih.gov/genome/tools/remap to normal chr start stop format

all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']
df = pd.read_csv("remapped_decipher_list_ucsc_format.txt", header=None, names=['chr', 'start', 'stop'], sep='\t')
df = df[df['chr'].isin(all_chr)] # only take rows with chromosomes in all_chr list
df = df.sort_index(by=['chr', 'start'])
df.to_csv('decipher_list_updated_hg38.txt', index=False, sep='\t',header=['chr', 'start', 'stop'])

/Users/stefanhosein/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [36]:
# sanity check to ensure that the variable regions is within the largest base pair size in hg38
df_d = pd.read_csv('decipher_list_updated_hg38.txt', sep='\t')
all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X']

all_sizes = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979,
             159345973, 145138636, 138394717, 133797422, 135086622, 133275309,
             114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
             58617616, 64444167, 46709983, 50818468, 156040895, 57227415]

for chrm, sizes in zip(all_chr, all_sizes):
    to_keep = [chrm]
    df_d_subset = df_d[df_d['chr'].isin(to_keep)]
    largest = df_d_subset.iloc[len(df_d_subset)-1,2]
    if largest > sizes:
        print 'Chrm is: ' + chrm + ' Size is: ' + str(sizes) + ' Largest: ' + str(largest)

In [37]:
df_d = pd.read_csv('decipher_list_updated_hg38.txt', sep='\t')
df_vr = pd.read_csv('variable_regions.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)

# list of choromozomes 
all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']

list_of_anomalies = [] # if and start and stop range within decipher is not fully within a VR range
list_of_non_anomlies = []

for chrm in all_chr:

    # choose chromosomes only in all_chr list since there maybe others
    to_keep = [chrm]


    df_d_subset = df_d[df_d['chr'].isin(to_keep)]
    df_vr_subset = df_vr[df_vr['chr'].isin(to_keep)]
    
    for i in xrange(len(df_d_subset)):

        # for each row get the start and stop point for VR in
        # VR text file

        start = df_d_subset.iloc[i, 1]
        stop = df_d_subset.iloc[i, 2]
        
        
        df_result = df_vr_subset[(df_vr_subset['start'] <= start) & (df_vr_subset['stop'] >= stop)]
        
        if df_result.empty: # this means that the decipher region is not completely within a VR
            list_of_anomalies.append([chrm, start, stop])
        
        df_overlap = df_vr_subset[(df_vr_subset['start'] <= start) & (df_vr_subset['stop'] >= stop)]

print len(list_of_anomalies)
print list_of_anomalies

3199
[['1', 4841648, 4972398], ['1', 5030655, 5113957], ['1', 5583698, 5587417], ['1', 6628412, 7376685], ['1', 7229254, 7229679], ['1', 7229634, 7229679], ['1', 7955376, 8244547], ['1', 8011717, 8012893], ['1', 8040080, 8091164], ['1', 11776626, 11778910], ['1', 11778639, 11778910], ['1', 11863925, 11911692], ['1', 11919444, 11920037], ['1', 12824272, 13388877], ['1', 12888219, 13454938], ['1', 13168699, 13413491], ['1', 13191047, 13454938], ['1', 13217533, 13393818], ['1', 13217533, 13385237], ['1', 13330100, 13396660], ['1', 13330100, 13393520], ['1', 13377112, 13384921], ['1', 13377519, 13384915], ['1', 13380367, 13384921], ['1', 13380504, 13384921], ['1', 13455628, 13463323], ['1', 14109507, 14112465], ['1', 14499162, 14501586], ['1', 15066799, 15385027], ['1', 15522041, 15544008], ['1', 15528989, 15529525], ['1', 16291191, 16438467], ['1', 17389749, 17400883], ['1', 17398584, 17398805], ['1', 19314095, 19334376], ['1', 19317539, 19329341], ['1', 19900606, 19916507], ['1', 2097287

In [39]:
# Take the values from variations_start_stop.txt and consolidate tha ranges and put this into variable_regions.txt
# REQUIREMENTS: list_of_anomalies (from above cell)

def merge_intervals(times):
    saved = list(times[0])
    for st, en in sorted([sorted(t) for t in times]):
        if st <= saved[1]:
            saved[1] = max(saved[1], en)
        else:
            yield tuple(saved)
            saved[0] = st
            saved[1] = en
    yield tuple(saved)


# read the text file
df = pd.read_csv('variations_start_stop.txt', sep='\t')
all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']

for chrm in all_chr:

    # choose chromosomes only in all_chr list since there maybe others
    to_keep = [chrm]
    # only keep rows that contain the string in to_keep
    df_subset = df[df['chr'].isin(to_keep)]
    # create intervals of the form: [(5, 7), (11, 116), (3, 4), (10, 12), (7, 8), (6, 12)]
    list_subset = []
    
    # get the subset from the list of anomalies that have the same chromosome number
    for item in list_of_anomalies:
        if item[0] == chrm:
            list_subset.append(item)

    for ls in list_subset:
        df_subset.loc[len(df_subset)] = ls
    
        
    intervals_list = []
    for i in xrange(len(df_subset)):
        intervals_list.append((df_subset.iloc[i,1], df_subset.iloc[i,2]))
#     print "done creating intervals"
    # run the function to get the merged values
    merged_list = merge_intervals(intervals_list)
#     print "done merging intervals"
    sorted_merged_list = sorted(merged_list, key=lambda tup: tup[0])
#     print "done sorting intervals"
    with open('variable_regions_update.txt', 'ab') as vr_file:
        for vr in sorted_merged_list:
            string = str(chrm) + '\t' + str(vr[0]) + '\t' + str(vr[1]) + '\n'
            vr_file.write(string)
    print "Finished Chromosome" + str(chrm)

Finished Chromosome1
Finished Chromosome2
Finished Chromosome3
Finished Chromosome4
Finished Chromosome5
Finished Chromosome6
Finished Chromosome7
Finished Chromosome8
Finished Chromosome9
Finished Chromosome10
Finished Chromosome11
Finished Chromosome12
Finished Chromosome13
Finished Chromosome14
Finished Chromosome15
Finished Chromosome16
Finished Chromosome17
Finished Chromosome18
Finished Chromosome19
Finished Chromosome20
Finished Chromosome21
Finished Chromosome22
Finished ChromosomeX
Finished ChromosomeY


/Users/stefanhosein/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [40]:
# take the values from variable_regions_updated.txt and create non_variable_regions_updated.txt where 
# the NVR file is disjoint

all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']
all_sizes = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979,
             159345973, 145138636, 138394717, 133797422, 135086622, 133275309,
             114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
             58617616, 64444167, 46709983, 50818468, 156040895, 57227415]

df = pd.read_csv('variable_regions_update.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)

for chrm, end in zip(all_chr, all_sizes):

    start = 1
    non_variable_regions = []

    to_keep = [chrm]
    # only keep rows that contain the string in to_keep
    df2 = df[df['chr'].isin(to_keep)]
    for i in xrange(len(df2)):
        # calculate the stop point, it is the base pair just before the start in the VR
        stop = df2.iloc[i, 1] - 1
        if start <= stop:
            non_variable_regions.append([start, stop])
        # calculate the start point, it is the base pair just after the stop in the VR
        start = df2.iloc[i, 2] + 1

    # need to determine if the last stop point is less than or equal to the total # of base pairs
    # if it is then this is a new NVR
    if start <= end:
        non_variable_regions.append([start, end])

    # save the file
    with open('non_variable_regions_update.txt', 'ab') as myfile:
        for non_variation in non_variable_regions:
            string = chrm + '\t' + str(non_variation[0]) + '\t' + str(non_variation[1]) + '\n'
            myfile.write(string)
    print "Finished Chromosome" + str(chrm)
    

Finished Chromosome1
Finished Chromosome2
Finished Chromosome3
Finished Chromosome4
Finished Chromosome5
Finished Chromosome6
Finished Chromosome7
Finished Chromosome8
Finished Chromosome9
Finished Chromosome10
Finished Chromosome11
Finished Chromosome12
Finished Chromosome13
Finished Chromosome14
Finished Chromosome15
Finished Chromosome16
Finished Chromosome17
Finished Chromosome18
Finished Chromosome19
Finished Chromosome20
Finished Chromosome21
Finished Chromosome22
Finished ChromosomeX
Finished ChromosomeY


In [44]:
# sanity check
all_sizes = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979,
             159345973, 145138636, 138394717, 133797422, 135086622, 133275309,
             114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
             58617616, 64444167, 46709983, 50818468, 156040895, 57227415]
total_bp = sum(all_sizes)
df1 = pd.read_csv('variable_regions_update.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
df2 = pd.read_csv('non_variable_regions_update.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
print total_bp
vr_bp = sum(df1['stop'] - df1['start'] + 1)
nvr_bp = sum(df2['stop'] - df2['start'] + 1)
print vr_bp
print nvr_bp
print vr_bp + nvr_bp

# further sanity check to ensure that each chromosome length is equivalent
# should not print any "Chro: 1 ..." if it works
all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']

for chrm, size in zip(all_chr, all_sizes):
    df1_sub = df1[df1['chr'] == chrm]
    df2_sub = df2[df2['chr'] == chrm]
    vr_bp = sum(df1_sub['stop'] - df1_sub['start'] + 1)
    nvr_bp = sum(df2_sub['stop'] - df2_sub['start'] + 1)
    sum_size = vr_bp + nvr_bp
    if sum_size != size:
        print " Chro: " + chrm + " Size should be: " + str(size) + " Sum is: " + str(sum_size)


3088269832
2461584143
626685689
3088269832


In [7]:
df = pd.read_csv('variable_regions.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
df_up = pd.read_csv('variable_regions_update.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
df_bp = sum(df['stop'] - df['start'] + 1)
df_up_bp = sum(df_up['stop'] - df_up['start'] + 1)
print 'Original: ' + str(df_bp)
print 'Updated: ' + str(df_up_bp)
print 'Difference: ' + str(df_up_bp - df_bp)

df = pd.read_csv('non_variable_regions.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
df_up = pd.read_csv('non_variable_regions_update.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
df_bp = sum(df['stop'] - df['start'] + 1)
df_up_bp = sum(df_up['stop'] - df_up['start'] + 1)
print 'Original: ' + str(df_bp)
print 'Updated: ' + str(df_up_bp)
print 'Difference: ' + str(df_bp - df_up_bp)

Original: 2402529413
Updated: 2461584143
Difference: 59054730
Original: 685740419
Updated: 626685689
Difference: 59054730
